# Setting Up

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
OPEN_API_KEY = os.getenv('OPEN_API_KEY')

LANGSMITH_TRACING = os.getenv('LANGSMITH_TRACING')
LANGSMITH_API_KEY = os.getenv('LANGSMITH_API_KEY')
LANGSMITH_PROJECT = os.getenv('LANGSMITH_PROJECT')
LANGSMITH_ENDPOINT = os.getenv('LANGSMITH_ENDPOINT')

TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Tools

# research tool

In [8]:
from tavily import TavilyClient
from pprint import pprint
tavily = TavilyClient(TAVILY_API_KEY)

In [5]:
def researcher(query):
    """Search the web for information"""
    response = tavily.search(query)
    return response

In [10]:
result = researcher("What is the capital of France?")
pprint(result)

{'answer': None,
 'follow_up_questions': None,
 'images': [],
 'query': 'What is the capital of France?',
 'response_time': 1.5,
 'results': [{'content': 'Paris is the capital city of France. Contrary to '
                         'popular belief, the name of the city did not come '
                         'from the Paris in Greek myths. Located in the north '
                         'of Central France, the city is relatively flat with '
                         'the highest point being 427 feet (which is '
                         'Montmartre) above sea level while the lowest point '
                         'is 115 feet above the sea level. With a GDP of '
                         'approximately $850 billion, Paris is the country’s '
                         'leader for economic activities. Tourists are '
                         'attracted to Paris because of the rich history the '
                         'city has including its monuments and sights. As the '
                    